# Quickstart

Runs through the API for common tasks in ML

### Working with data

Pytorch has two primitives to work with data:
- torch.utils.data.DataLoader
- torch.utils.data.Dataset

Dataset stores the samples and their corresponding labels. Dataloader wraps an iterable around the Dataset

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

PyTorch offers domain-specific libraries such as TorchText, TorchVision, and TorchAudio, all of which include datasets. 

For this tutorial, we will be using a TorchVision dataset.

The torchvision.datasets module contains Dataset objects for many real-world vision data like CIFAR, COCO, etc (full list: https://pytorch.org/vision/stable/datasets.html). 

In this tutorial, we use the FashionMNIST dataset. Every TorchVision Dataset includes two arguments: transform and target_transform to modify the samples and labels respectively.

In [5]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

Pass the Dataset as an argument to DataLoader. This wraps an iterable over our dataset, and supports automatic batching, sampling, shuffling and multiprocess data loading. 

Here we define a batch size of 64, i.e. each element in the dataloader iterable will return a batch of 64 features and labels.

In [6]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


N: This represents the number of samples or examples in the batch. In the context of image data, N would be the batch size, which is the number of images processed together during one iteration of training or inference.

C: This represents the number of channels in the image. For grayscale images, C would be 1, indicating a single channel. For color images represented in RGB format, C would be 3, representing the three color channels (red, green, blue).

H: This represents the height of the image in pixels.

W: This represents the width of the image in pixels.

### Creating Models

In [7]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


In [8]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Above code defines a neural network model using the nn.Module class and prints out a summary of the model architecture.

1. Define the Neural Network Class (NeuralNetwork):

    The NeuralNetwork class is defined, which is a subclass of nn.Module, the base class for all neural network modules in PyTorch.

    The __init__ method is called when an instance of the class is created. Inside __init__, the layers of the neural network are defined.
    

2. Define Layers in the __init__ Method:

    - The __init__ method initializes the neural network layers:
        - nn.Flatten(): This layer flattens the input tensor into a 1D tensor. It is used to flatten the input images, which are 2D (28x28 pixels), into a 1D tensor (784 pixels).
        - nn.Sequential(): This is a container that sequentially applies a list of layers.
        - nn.Linear(): This defines fully connected (dense) layers. It takes the size of the input and output as parameters.
        - nn.ReLU(): This is the rectified linear unit (ReLU) activation function, applied after each linear layer except the last one.
    - The neural network consists of three linear layers with ReLU activation functions between them.
    - The first linear layer takes 28x28 input features (the size of the flattened image) and outputs 512 features.
    - The second and third linear layers have 512 input features and output 512 features, and 10 features respectively.
    - The output of the last linear layer (with 10 output features) represents the logits for each class (in this case, the FashionMNIST dataset has 10 classes).


3. Define the forward Method:

    - The forward method defines the forward pass of the neural network.
    - It takes an input tensor x and passes it through the layers defined in the __init__ method.
    - First, the input tensor is flattened using nn.Flatten().
    - Then, the flattened tensor is passed through the sequential layer defined in the __init__ method.
    - The output of the last linear layer (logits) is returned.
    
    
4. Instantiate the Model and Move to Device (model = NeuralNetwork().to(device)):

    - An instance of the NeuralNetwork class is created.
    - It is then moved to the device specified earlier (CPU, GPU, or MPS) using the to method.

### Optimizing the Model Parameters